# Fermi's Method for pun detection


In [30]:
import string
import nltk
from nltk.corpus import wordnet
import torch
import torch.nn as nn
import torch. optim as optim
import torch.nn.functional as F
import bcolz
import numpy as np
import pickle

import xml.etree.ElementTree as ET
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
puncts = string.punctuation
tree = ET.parse('../data/test/subtask1-homographic-test.xml')

# Prepare Dataset

In [9]:
def make_dataset():
    F = open("../data/test/subtask1-homographic-test.gold")
    scores = list()
    sents = list()
    root = tree.getroot()
    for item in F.readlines():
        ans = item.split()[1]
        scores.append(ans)
    for text in root.getchildren():
        sent = list()
        for word in text.getchildren():
            sent.append(word.text)
        sents.append(sent)
    return sents, scores

sentences, scores = make_dataset()

In [11]:
count = 0
train_sent = list()
train_score = list()
test_sent = list()
test_score = list()

for index, sentence in enumerate(sentences):
    if((index + 1) / 2250 < 0.8): # 80-20 split for training and testing
        train_sent.append(sentence)
        train_score.append(scores[index])
    else:
        test_sent.append(sentence)
        test_score.append(scores[index])

In [10]:
def filter_sent(sent):
    sent = [w for w in sent if w.lower() not in stopwords]
    sent = [w for w in sent if w[0] not in puncts]
    return sent

In [16]:
clean_sentences_train = list()
for sentence in train_sent:
    sentence = filter_sent(sentence)
    clean_sentences_train.append(sentence)

In [ ]:
matrix_len = len(target_vocab)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

In [31]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'../../../temp/glove.6B/glove.6B.50d.dat', mode='w')

with open(f'../../../temp/glove.6B/glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400001, 50)), rootdir=f'../../../temp/glove.6B/glove.6B.50d.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'../../../temp/glove.6B/6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'../../../temp/glove.6B/6B.50_idx.pkl', 'wb'))

In [34]:
vectors = bcolz.open(f'../../../temp/glove.6B/glove.6B.50d.dat/')[:]
words = pickle.load(open(f'../../../temp/glove.6B/6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'../../../temp/glove.6B/6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [37]:
glove['ant']

array([ 0.6734   , -0.48774  , -0.5861   ,  0.32338  ,  0.56756  ,
        0.23663  , -0.64305  , -0.53961  , -0.0099409, -0.80632  ,
        0.35767  ,  0.9699   ,  0.67127  ,  0.40689  ,  0.1089   ,
        0.35832  ,  0.74568  ,  0.28874  , -1.1524   ,  0.41785  ,
       -0.42827  , -0.63627  ,  0.60103  ,  0.97918  ,  0.087392 ,
        0.20913  ,  0.079497 , -0.14906  ,  0.47589  ,  0.14898  ,
        0.62786  , -1.0231   ,  0.034738 ,  0.2563   , -0.48078  ,
        0.52082  , -0.15008  , -1.1917   ,  0.19115  , -0.15885  ,
        0.28865  , -0.59585  , -0.60087  ,  0.61836  ,  1.7035   ,
       -0.041247 ,  0.26282  , -0.80837  ,  0.42369  ,  0.10184  ])

In [ ]:
class Encoder(nn.Module):
    def __init__(self, hidden_dim, batch_size, n_layers, vocab_size):
        super(Encoder).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        
        self.embedding = nn.Embedding(self.vocab_size, self.hidden_dim)
        self.lstm = nn.LSTMCell(self.batch_size, self.hidden_dim, n_layers, bidirectional=True)
        
        self.lin = nn.Linear( , 1) # fill
        self.sig = nn.Sigmoid()
        
    def forward(self, inp, hidden):
        embed = self.embedding(inp)
        op, _ = self.lstm(embed, hidden)
        op = (op[:,:,:self.hidden_dim] + op[:,:,self.hidden_dim:]) # bidirectional outputs
        op = self.lin(op)
        op = self.sig(op)
        return op